# Basic Conversion Functions and Frequency Analysis

In [46]:
from string import ascii_lowercase
from math import ceil
import numpy as np

# construct a dictionary with the standard letter->integer map. 
# change start=0 to start=1 if you want a->1, b->2, ...
strconvert = {letter: index for index, letter in enumerate(ascii_lowercase, start=0)}

# construct the inverse dictionary
numconvert = {index: letter for letter,index in strconvert.items()}

def str2num(text):

    # input: a string (may contain non-alphabetic characters)
    # output: a corresponding array of integers under the a->0, b->1, ... map
    #         note that any non-alphabetic characters (e.g. punctuation) will be skipped
    # example: str2num('bad') = [1,0,3]

    # make sure all text is lowercase to match the strconvert dictionary
    text = text.lower()

    # perform the conversion. note that any non-alphabetical characters will be skipped
    numbers = [strconvert[letter] for letter in text if letter in strconvert]
    return numbers
  
def num2str(numbers):

    # input: an array of integers
    # output: a corresponding string under the 0->a, 2->b map

    # perform the conversion. note that any numbers outside the 0-25 range will be ignored
    letters = [numconvert[number] for number in numbers if number in numconvert]

    string = ''.join(letters)

    return string

def frequency(text):
      # input: a ciphertext
      # output: a dictionary containing every letter in the ciphertext and its frequency in the ciphertext

      # convert text to numbers to use numpy functions
      numeric_text = np.array(str2num(text))

      # get all unique values in the numeric text and the number of times each occurs
      unique,counts = np.unique(numeric_text,return_counts=True)
      freqs = np.round(counts / numeric_text.size,3)

      # get a list of unique characters for ease of reading output
      unique_chars = list(num2str(unique))

      # construct a dictionary of letters and their frequencies
      frequencies = list(zip(unique_chars,freqs))

      # sort by highest to lowest
      frequencies = sorted(frequencies,key = lambda x: x[1],reverse=True)
      return frequencies

# Shift and Affine Ciphers

In [47]:
def affine(text,scale,shift):
  
    # input: a string and the key for an affine cipher
    # output: the encrypted/decrypted string

    # convert text to numbers
    numbers = np.array(str2num(text))

    # perform the modular arithmetic
    numbers = np.remainder(scale*numbers + shift,26)

    # convert back to text
    text = num2str(list(numbers))

    return text

def shift(text,shift):
    # input: a string and the key for a shift cipher
    # output: the encrypted/decrypted string

    # convert text to numbers
    numbers = np.array(str2num(text))

    # perform the shift
    numbers = np.remainder(numbers+shift,26)

    # return the encrypted/decrypted text
    text = num2str(list(numbers))
    return text

# Vigenere encryption and attacks

In [ ]:
def vigenere(text,key,encrypt=True):
    # input: text and key for vigenere encryption. to decrypt set encrypt=False
    # output: the encrypted or decrypted text

    # convert text and key to numbers
    numeric_text = np.array(str2num(text))
    numeric_key = np.array(str2num(key))

    # get text and key length to repeat the key
    text_length = numeric_text.size
    key_length = numeric_key.size

    # extend the key to the length of the text
    numeric_key = np.tile(numeric_key,ceil(text_length/key_length))[:text_length]

    # perform the encryption or decryption
    if encrypt:
      numeric_output = np.remainder(numeric_text + numeric_key,26)
    else:
      numeric_output = np.remainder(numeric_text - numeric_key,26)
    
    text_output = num2str(numeric_output)

    return text_output

In [ ]:
def coincidence(ciphertext,limit=10):
    # note that this is not the "index of coincidence"
    # this counts coincidences between shifted versions of the ciphertext to help guess key length

    # input: a ciphertext believed to be encrypted using Vigenere
    # output: coincidence counts for shifts of 1 up to limit

    numeric_ciphertext = np.array(str2num(ciphertext))

    # initialize a shifted version of the ciphertext
    shift = numeric_ciphertext

    # initialize an array to story the output
    coincidences = np.zeros(limit)

    for i in range(limit):
      
      # generate a "shifted" version of the ciphertext by inserting -1 at the beginning, since -1 is never in the ciphertext
      # note that we then truncate so it is the same length as the original text
      shift = np.insert(shift,0,-1)[:numeric_ciphertext.size]

      # count the number of coincidences at this shift
      coincidences[i] = np.sum(numeric_ciphertext==shift)
    
    return coincidences

# Examples

## Shift Cipher
First, we receive the following encrypted message

In [48]:
ciphertext = 'nkdtz ymymw tzlmt zyfqq mnxyt wdmfi mfifh mfrun tsytx yfsiz uktwn yytxm tbfit zgyns lbtwq iymfy fhmnq ihfsy mnspf siutx xngqd itnyu wfhyn hfqqd dtzbt zqisy htsxy fsyqd wzsfh wtxxk tqpxy tifdb mthqf nrymf yfhmn qiits ypstb fsdym nslfh mnqix gwfns xyfwy xkzsh yntsn slfyg nwymf simfx frtsl xynyx rfsdn skfsy htsat qzynt sxymt zxfsi xtkit wrfsy fytrx nsytb mnhml timfx uzyfr dxynh utxxn gnqny dktws tynhn slfsf izqyx fhyfs iknlz wnslt zynyx uzwut wy'

You suspect a shift cipher and perform a frequency analysis

In [50]:
frequency(ciphertext)

[('y', 0.118),
 ('f', 0.102),
 ('t', 0.102),
 ('s', 0.089),
 ('n', 0.086),
 ('x', 0.067),
 ('m', 0.059),
 ('i', 0.051),
 ('h', 0.043),
 ('q', 0.043),
 ('z', 0.043),
 ('w', 0.04),
 ('d', 0.03),
 ('l', 0.024),
 ('k', 0.022),
 ('u', 0.022),
 ('r', 0.019),
 ('b', 0.016),
 ('g', 0.013),
 ('p', 0.008),
 ('a', 0.003)]

Since "y" is the most common character, you guess that plaintext "e" corresponds to ciphertext "y", and calculate the decryption key

In [54]:
y = str2num('y')[0]   # get the integer for y
e = str2num('e')[0]   # get the integer for e
key = y - e           # get the shift amount to turn "e" into "y"

20

Unfortunately, decryption produces nothing intelligible

In [57]:
shift(ciphertext,-key)  # decrypt using -key to shift backwards

'tqjzfesesczfrszfelwwstdezcjsloslolnslxatzyezdelyofaqzcteezdszhlozfmetyrhzcwoeslelnstwonlyestyvlyoazddtmwjozteaclnetnlwwjjzfhzfwoyenzydelyewjcfylnczddqzwvdezoljhsznwltxeslelnstwoozyevyzhlyjestyrlnstwodmcltydelcedqfynetzytyrlemtceslyosldlxzyrdetedxlyjtyqlyenzygzwfetzydeszfdlyodzqozcxlyelezxdtyezhstnsrzosldafelxjdetnazddtmtwtejqzcyzetntyrlylofwedlnelyoqtrfctyrzfetedafcazce'

So maybe ciphertext "y" corresponds to another frequent letter, say "t". Testing that seems to work! At least, it produces what looks like an English paragraph.

In [58]:
t = str2num('t')[0]
key = y-t
shift(ciphertext,-key)

'ifyouththroughoutallhistoryhadhadachampiontostandupforittoshowadoubtingworldthatachildcanthinkandpossiblydoitpracticallyyouwouldntconstantlyrunacrossfolkstodaywhoclaimthatachilddontknowanythingachildsbrainstartsfunctioningatbirthandhasamongstitsmanyinfantconvolutionsthousandsofdormantatomsintowhichgodhasputamysticpossibilityfornoticinganadultsactandfiguringoutitspurport'

Googling this phrase ("if youth throughout all history had had a champion...") leads you to believe this is an excerpt from the book Gadsby -- a book containing no letter "e", specifically to annoy cryptography students!